# Coefficient Comparison with Statistical Analysis: Config vs Estimated (Tables 1-24)

This notebook compares the coefficients from the configuration file (ground truth) with the estimated coefficients from the JSON output for **all 24 tables**, now enhanced with **comprehensive statistical analysis**.

**Features:**
- **Complete Coverage**: All tables 1-24 with their respective model types
- **Config Order Preservation**: Variables displayed in the same order as they appear in the config file
- **Clean Comparison Tables**: Variable names, config values, estimated values, sign matches, and magnitude differences
- **Enhanced Statistical Analysis**: 
  - **t-test Statistics**: Std. Error, CI Lower, CI Upper, Config in CI (for level models)
  - **Chi-square Statistics**: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05) (for probability models)
  - **Automatic Detection**: Detects statistical format and displays appropriate measures
- **Special Handling**: Intercept lists, multinomial models, and various model specifications
- **Auto-Detection**: Automatically detects model types when configurations are incomplete
- **Comprehensive Analysis**: Covers economic depreciation, assets, liabilities, income, expenses, and financial metrics

**Output Format:**
- ✅/❌ Sign agreement indicators
- Magnitude differences in log orders
- **Statistical measures** displayed alongside coefficient comparisons
- Summary statistics for each model including statistical data availability
- Success rate tracking across all tables

**Statistical Integration:**
- **Level Models** → t-test format: Standard errors, confidence intervals, and coverage indicators
- **Probability Models** → Chi-square format: Test statistics, p-values, and significance indicators (p<0.05)
- **Robust Handling**: Works with or without statistical data in JSON files

## 1. Import Libraries and Setup

In [17]:
import pandas as pd
import numpy as np
import json
import sys
import glob

# Add the estimators directory to the path to import config
sys.path.append('estimators/configs')

# Configure display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 6)
pd.set_option('display.width', 1000)

print("✅ Libraries imported successfully!")
print("📁 Config path added to sys.path")
print("🔧 Display options configured")

✅ Libraries imported successfully!
📁 Config path added to sys.path
🔧 Display options configured


## 2. Core Comparison Functions

In [18]:
def compare_coefficients(config_dict, config_step_name, estimated_json_path, model_name="Model"):
    """
    Compare coefficients between a config step and estimated JSON output.
    
    Parameters:
    - config_dict: Dictionary containing config steps
    - config_step_name: Name of the step within config_dict to compare  
    - estimated_json_path: Path to the JSON file with estimated coefficients
    - model_name: Display name for the model
    
    Returns:
    - DataFrame with comparison results, or None if error
    
    Key Features:
    - Preserves the order of variables as they appear in the config file
    - Special handling for Intercept lists (splits into Intercept1, Intercept2, etc.)
    - Handles missing variables gracefully
    - Includes statistical analysis (t-test or Chi-square) when available
    """
    
    # Find the config step
    config_step = None
    for step in config_dict['steps']:
        if step['name'] == config_step_name:
            config_step = step
            break
    
    if config_step is None:
        print(f"❌ Config step '{config_step_name}' not found!")
        available_steps = [s['name'] for s in config_dict['steps']]
        print(f"   Available steps: {available_steps}")
        return None
    
    config_coef = config_step['coefficients']
    
    # Load estimated coefficients
    try:
        with open(estimated_json_path, 'r') as f:
            estimated_data = json.load(f)
        
        # Handle different JSON structures
        if 'coefficients' in estimated_data:
            estimated_coef = estimated_data['coefficients']
            statistics_data = estimated_data.get('statistics', {})
        else:
            estimated_coef = estimated_data
            statistics_data = {}
            
    except FileNotFoundError:
        print(f"❌ Estimated file not found: {estimated_json_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Error parsing JSON file {estimated_json_path}: {e}")
        return None
    
    # Process statistics data into a dictionary for easy lookup
    stats_lookup = {}
    if statistics_data:
        # Handle different statistics formats
        if 'coefficient_stats' in statistics_data:
            # Chi-square format (probability models)
            for stat in statistics_data['coefficient_stats']:
                feature = stat.get('feature')
                if feature:
                    stats_lookup[feature] = {
                        'std_error': stat.get('Std. Error'),
                        'test_statistic': stat.get('Chi-square'),
                        'p_value': stat.get('Pr(>ChiSq)'),
                        'test_type': 'chi_square'
                    }
        elif isinstance(statistics_data, list):
            # t-test format (level models)
            for stat in statistics_data:
                feature = stat.get('feature')
                if feature:
                    stats_lookup[feature] = {
                        'std_error': stat.get('Std. Error'),
                        'ci_lower': stat.get('CI Lower'),
                        'ci_upper': stat.get('CI Upper'),
                        't_statistic': stat.get('t-statistic'),
                        'p_value': stat.get('p-value'),
                        'test_type': 't_test'
                    }
    
    # Create comparison DataFrame - PRESERVE CONFIG ORDER by iterating through config_coef.items()
    comparison_data = []
    
    # Keep track of order for proper indexing
    config_order = 0
    
    for var_name, config_value in config_coef.items():
        # Special handling for Intercept lists (e.g., [-2.6765, -1.1358])
        if var_name == "Intercept" and isinstance(config_value, list):
            # Handle Intercept list by mapping to Intercept1, Intercept2, etc.
            for i, intercept_val in enumerate(config_value, 1):
                intercept_var_name = f"Intercept{i}"
                estimated_val = estimated_coef.get(intercept_var_name)
                stats = stats_lookup.get(intercept_var_name, {})
                
                comparison_data.append({
                    'config_order': config_order + i - 1,  # Maintain sub-order for intercepts
                    'variable': intercept_var_name,
                    'config': intercept_val,
                    'estimated': estimated_val if estimated_val is not None else np.nan,
                    'is_missing': estimated_val is None,
                    'stats': stats
                })
            config_order += len(config_value)  # Increment by number of intercepts
            continue
        
        # Handle other list values by taking the first element
        if isinstance(config_value, list):
            if len(config_value) > 0:
                config_value = config_value[0]  # Take first element
            else:
                config_value = 0.0  # Default for empty lists
        
        # Check if variable exists in estimated
        stats = stats_lookup.get(var_name, {})
        if var_name in estimated_coef:
            estimated_val = estimated_coef[var_name]
            comparison_data.append({
                'config_order': config_order,
                'variable': var_name,
                'config': config_value,
                'estimated': estimated_val,
                'is_missing': False,
                'stats': stats
            })
        else:
            # Variable exists in config but missing in estimated
            comparison_data.append({
                'config_order': config_order,
                'variable': var_name,
                'config': config_value,
                'estimated': np.nan,
                'is_missing': True,
                'stats': stats
            })
        
        config_order += 1
    
    # Create DataFrame and sort by config_order to preserve original order
    df = pd.DataFrame(comparison_data)
    df = df.sort_values('config_order').reset_index(drop=True)
    
    # Add derived columns for analysis
    df['config_sign'] = np.sign(df['config'])
    df['estimated_sign'] = df['estimated'].apply(lambda x: np.sign(x) if not pd.isna(x) else np.nan)
    df['same_sign'] = df.apply(lambda row: 
                              (row['config_sign'] == row['estimated_sign']) if not row['is_missing'] 
                              else np.nan, axis=1)
    
    # Calculate log magnitude differences
    epsilon = 1e-20
    df['config_abs'] = np.abs(df['config'])
    df['estimated_abs'] = df['estimated'].apply(lambda x: np.abs(x) if not pd.isna(x) else np.nan)
    df['config_log'] = np.log10(df['config_abs'] + epsilon)
    df['estimated_log'] = df['estimated'].apply(lambda x: np.log10(np.abs(x) + epsilon) if not pd.isna(x) else np.nan)
    df['log_diff'] = df.apply(lambda row: 
                             (row['estimated_log'] - row['config_log']) if not row['is_missing'] 
                             else np.nan, axis=1)
    
    return df

def display_comparison_table(comparison_df, model_name):
    """
    Display a clean comparison table for a single model with statistical analysis.
    Variables are shown in the same order as they appear in the config file.
    Includes statistical measures: t-test (Std. Error, CI Lower, CI Upper) or 
    Chi-square (Std. Error, Chi-square, Pr(>ChiSq)).
    """
    if comparison_df is None:
        return
    
    # Determine if we have statistical data and what type
    has_stats = any(comparison_df['stats'].apply(lambda x: bool(x)))
    stat_type = None
    if has_stats:
        # Find the first non-empty stats to determine type
        for stats in comparison_df['stats']:
            if stats:
                stat_type = stats.get('test_type', 'unknown')
                break
    
    print(f"📊 {model_name.upper()} - Coefficient Comparison:")
    print(f"   Variables ordered as they appear in config file")
    if has_stats:
        if stat_type == 't_test':
            print(f"   📈 Including t-test statistics: Std. Error, CI Lower, CI Upper, Config in CI")
        elif stat_type == 'chi_square':
            print(f"   📈 Including Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)")
        else:
            print(f"   📈 Including available statistical measures")
    print()
    
    # Format the data for display
    display_data = comparison_df.copy()
    
    # Format numeric values for display
    def format_scientific(val):
        if pd.isna(val):
            return "NA"
        try:
            val = float(val)
            if abs(val) < 1e-3 or abs(val) > 1e6:
                return f"{val:.3e}"
            else:
                return f"{val:.6f}"
        except:
            return str(val)
    
    def format_p_value(val):
        if pd.isna(val):
            return "NA"
        try:
            val = float(val)
            if val < 0.001:
                return f"{val:.3e}"
            else:
                return f"{val:.6f}"
        except:
            return str(val)
    
    # Create display columns
    display_data['Config Value'] = display_data['config'].apply(format_scientific)
    display_data['Estimated Value'] = display_data.apply(lambda row: 
                                                        "NA" if row['is_missing'] 
                                                        else format_scientific(row['estimated']), axis=1)
    
    # Sign match column
    display_data['Sign Match'] = display_data.apply(lambda row: 
                                                   "NA" if row['is_missing']
                                                   else ("✅ Same" if row['same_sign'] else "❌ Different"), axis=1)
    
    # Magnitude difference column
    display_data['Magnitude Diff'] = display_data.apply(lambda row: 
                                                       "NA" if row['is_missing']
                                                       else f"{row['log_diff']:+.1f} orders", axis=1)
    
    # Add statistical columns based on available data
    columns_to_show = ['Variable', 'Config Value', 'Estimated Value', 'Sign Match', 'Magnitude Diff']
    
    if has_stats:
        if stat_type == 't_test':
            # Add t-test specific columns
            display_data['Std. Error'] = display_data.apply(lambda row: 
                format_scientific(row['stats'].get('std_error')) if row['stats'] else "NA", axis=1)
            display_data['CI Lower'] = display_data.apply(lambda row: 
                format_scientific(row['stats'].get('ci_lower')) if row['stats'] else "NA", axis=1)
            display_data['CI Upper'] = display_data.apply(lambda row: 
                format_scientific(row['stats'].get('ci_upper')) if row['stats'] else "NA", axis=1)
            
            # Add confidence interval coverage indicator for t-test models
            display_data['Config in CI'] = display_data.apply(lambda row: 
                "✅ Yes" if (row['stats'] and 
                           row['stats'].get('ci_lower') is not None and 
                           row['stats'].get('ci_upper') is not None and
                           not pd.isna(row['stats'].get('ci_lower')) and
                           not pd.isna(row['stats'].get('ci_upper')) and
                           not row['is_missing'] and
                           float(row['stats'].get('ci_lower')) <= row['config'] <= float(row['stats'].get('ci_upper'))) 
                else ("❌ No" if (row['stats'] and 
                                row['stats'].get('ci_lower') is not None and 
                                row['stats'].get('ci_upper') is not None and
                                not pd.isna(row['stats'].get('ci_lower')) and
                                not pd.isna(row['stats'].get('ci_upper')) and
                                not row['is_missing'])
                      else "NA"), axis=1)
            
            columns_to_show.extend(['Std. Error', 'CI Lower', 'CI Upper', 'Config in CI'])
            
        elif stat_type == 'chi_square':
            # Add Chi-square specific columns
            display_data['Std. Error'] = display_data.apply(lambda row: 
                format_scientific(row['stats'].get('std_error')) if row['stats'] else "NA", axis=1)
            display_data['Chi-square'] = display_data.apply(lambda row: 
                format_scientific(row['stats'].get('test_statistic')) if row['stats'] else "NA", axis=1)
            display_data['Pr(>ChiSq)'] = display_data.apply(lambda row: 
                format_p_value(row['stats'].get('p_value')) if row['stats'] else "NA", axis=1)
            
            # Add significance indicator for Chi-square tests
            display_data['Significant'] = display_data.apply(lambda row: 
                "✅ Yes" if (row['stats'] and 
                           row['stats'].get('p_value') is not None and 
                           not pd.isna(row['stats'].get('p_value')) and 
                           float(row['stats'].get('p_value')) < 0.05) 
                else ("❌ No" if (row['stats'] and 
                                row['stats'].get('p_value') is not None and 
                                not pd.isna(row['stats'].get('p_value')))
                      else "NA"), axis=1)
            
            columns_to_show.extend(['Std. Error', 'Chi-square', 'Pr(>ChiSq)', 'Significant'])
    
    # Create clean display table with proper column order
    final_table = display_data[['variable'] + columns_to_show[1:]].copy()
    final_table.columns = columns_to_show
    
    # Display the table (already in config order)
    display(final_table)
    
    # Summary statistics
    total_vars = len(comparison_df)
    missing_vars = comparison_df['is_missing'].sum()
    found_vars = total_vars - missing_vars
    sign_matches = comparison_df['same_sign'].sum() if found_vars > 0 else 0
    stats_available = sum(1 for stats in comparison_df['stats'] if stats)
    
    print(f"📊 Summary:")
    print(f"   • {found_vars}/{total_vars} variables found in estimated output")
    print(f"   • {sign_matches}/{found_vars} variables have matching signs" if found_vars > 0 else "   • No variables found for sign comparison")
    if has_stats:
        print(f"   • {stats_available}/{found_vars} variables have statistical data ({stat_type.replace('_', '-')} format)")
        # Add significance summary for Chi-square tests
        if stat_type == 'chi_square':
            significant_vars = sum(1 for stats in comparison_df['stats'] 
                                 if (stats and stats.get('p_value') is not None and 
                                     not pd.isna(stats.get('p_value')) and 
                                     float(stats.get('p_value')) < 0.05))
            print(f"   • {significant_vars}/{stats_available} variables are significant (p<0.05)")
        # Add confidence interval coverage summary for t-test models
        elif stat_type == 't_test':
            covered_vars = sum(1 for i, row in comparison_df.iterrows() 
                             if (row['stats'] and 
                                 row['stats'].get('ci_lower') is not None and 
                                 row['stats'].get('ci_upper') is not None and
                                 not pd.isna(row['stats'].get('ci_lower')) and
                                 not pd.isna(row['stats'].get('ci_upper')) and
                                 not row['is_missing'] and
                                 float(row['stats'].get('ci_lower')) <= row['config'] <= float(row['stats'].get('ci_upper'))))
            print(f"   • {covered_vars}/{stats_available} config values are within confidence intervals")
    if missing_vars > 0:
        print(f"   ⚠️  {missing_vars} variables missing from estimated output")
    print()

print("✅ Enhanced comparison functions loaded!")
print("📈 Statistical analysis support added:")
print("   • t-test statistics: Std. Error, CI Lower, CI Upper, Config in CI")
print("   • Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)")
print("🎯 Variables will be displayed in config file order")
print("🔧 Special handling for Intercept lists included")

✅ Enhanced comparison functions loaded!
📈 Statistical analysis support added:
   • t-test statistics: Std. Error, CI Lower, CI Upper, Config in CI
   • Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)
🎯 Variables will be displayed in config file order
🔧 Special handling for Intercept lists included


## 3. Table Configuration and Comprehensive Analysis

In [19]:
def get_config_info(table_num):
    """Get the config module and variable name for each table"""
    config_info = {
        1: ('t1_edepma_config', 'EDEPMA_CONFIG'),
        2: ('t2_sma_config', 'SMA_CONFIG'),
        3: ('t3_ima_config', 'IMA_CONFIG'), 
        4: ('t4_edepbu_config', 'EDEPBU_CONFIG'),
        5: ('t5_ibu_config', 'IBU_CONFIG'),
        6: ('t6_dofa_config', 'DOFA_CONFIG'),
        7: ('t7_dca_config', 'DCA_CONFIG'),
        8: ('t8_dll_config', 'DLL_CONFIG'),
        9: ('t9_dcl_config', 'DCL_CONFIG'),
        10: ('t10_dsc_config', 'DSC_CONFIG'),
        11: ('t11_drr_config', 'DRR_CONFIG'),
        12: ('t12_oibd_config', 'OIBD_CONFIG'),
        13: ('t13_fi_config', 'FI_CONFIG'),
        14: ('t14_fe_config', 'FE_CONFIG'),
        15: ('t15_tdepma_config', 'TDEPMA_CONFIG'),
        16: ('t16_zpf_config', 'ZPF_CONFIG'),
        17: ('t17_dour_config', 'DOUR_CONFIG'),
        18: ('t18_gc_config', 'GC_CONFIG'),
        19: ('t19_oa_config', 'OA_CONFIG'),
        20: ('t20_tl_config', 'TL_CONFIG'),
        21: ('t21_ota_config', 'OTA_CONFIG'),
        22: ('t22_tdepbu_config', 'TDEPBU_CONFIG'),
        23: ('t23_pallo_config', 'PALLO_CONFIG'),
        24: ('t24_rot_config', 'ROT_CONFIG')
    }
    return config_info.get(table_num)

def get_table_info(table_num):
    """Get table information including model types and step mappings"""
    table_info = {
        1: {
            'name': 'EDEPMA (Economic Depreciation MA)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        2: {
            'name': 'SMA (Selling & Marketing Assets)',
            'models': [
                ('probability_model', 'Multinomial Probability', '_prob'),
                ('positive_level_model', 'Level Positive', '_level_pos'),
                ('negative_level_model', 'Level Negative', '_level_neg')
            ]
        },
        3: {
            'name': 'IMA (Investment Model Assets)', 
            'models': [('investment_model', 'Tobit Investment', '_level')]
        },
        4: {
            'name': 'EDEPBU (Economic Depreciation BU)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        5: {
            'name': 'IBU (Investment in Business Use)', 
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        6: {
            'name': 'DOFA (Disposal of Fixed Assets)',
            'models': [
                ('positive_probability_model', 'Probability Positive', '_prob_pos'),
                ('negative_probability_model', 'Probability Negative', '_prob_neg'),
                ('level_model', 'Level', '_level_neg')
            ]
        },
        7: {
            'name': 'DCA (Debt Capital Assets)',
            'models': [('level_model', 'Level', '_level')]
        },
        8: {
            'name': 'DLL (Debt Long-term Liabilities)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        9: {
            'name': 'DCL (Debt Current Liabilities)', 
            'models': [('level_model', 'Level', '_level')]
        },
        10: {
            'name': 'DSC (Debt Service Charges)',
            'models': [
                ('positive_probability_model', 'Probability Positive', '_prob_pos'),
                ('negative_probability_model', 'Probability Negative', '_prob_neg'),
                ('positive_level_model', 'Level Positive', '_level_pos'),
                ('negative_level_model', 'Level Negative', '_level_neg')
            ]
        },
        11: {
            'name': 'DRR (Debt Reduction/Repayment)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        12: {
            'name': 'OIBD (Operating Income Before Depreciation)',
            'models': [('level_model', 'Level', '_level')]
        },
        13: {
            'name': 'FI (Financial Income)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        14: {
            'name': 'FE (Financial Expenses)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        15: {
            'name': 'TDEPMA (Tax Depreciation MA)',
            'models': [('investment_model', 'Tobit Investment', '_prob')]
        },
        16: {
            'name': 'ZPF (Zero-Profit Firms)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        17: {
            'name': 'DOUR (Dividend Outflows/Unrestricted)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('positive_level_model', 'Level Positive', '_level_pos'),
                ('negative_level_model', 'Level Negative', '_level_neg')
            ]
        },
        18: {
            'name': 'GC (Government Contributions)',
            'models': [
                ('positive_probability_model', 'Probability Positive', '_prob_pos'),
                ('negative_probability_model', 'Probability Negative', '_prob_neg'),
                ('positive_level_model', 'Level Positive', '_level_pos'),
                ('negative_level_model', 'Level Negative', '_level_neg')
            ]
        },
        19: {
            'name': 'OA (Other Assets)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('positive_level_model', 'Level Positive', '_level_pos'),
                ('negative_level_model', 'Level Negative', '_level_neg')
            ]
        },
        20: {
            'name': 'TL (Total Liabilities)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        },
        21: {
            'name': 'OTA (Other Assets/Total)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('positive_level_model', 'Level Positive', '_level_pos'),
                ('negative_level_model', 'Level Negative', '_level_neg')
            ]
        },
        22: {
            'name': 'TDEPBU (Tax Depreciation BU)',
            'models': []  # Will be determined by checking available files
        },
        23: {
            'name': 'PALLO (Provision Allowances)',
            'models': [('level_model', 'Level', '_level')]
        },
        24: {
            'name': 'ROT (Return on Total Assets)',
            'models': [
                ('probability_model', 'Probability', '_prob'),
                ('level_model', 'Level', '_level')
            ]
        }
    }
    return table_info.get(table_num)

def find_estimated_files(table_num):
    """Find all estimated coefficient files for a table"""
    pattern = f"estimators/coef/t{table_num}_*"
    files = glob.glob(pattern)
    return sorted(files)

def auto_detect_models(table_num, estimated_files):
    """Auto-detect model types based on available estimated files"""
    model_types = []
    
    # Common patterns for model detection
    patterns = [
        ('_prob.json', 'Probability', '_prob'),
        ('_level.json', 'Level', '_level'),
        ('_prob_pos.json', 'Probability Positive', '_prob_pos'),
        ('_prob_neg.json', 'Probability Negative', '_prob_neg'),
        ('_level_pos.json', 'Level Positive', '_level_pos'),
        ('_level_neg.json', 'Level Negative', '_level_neg'),
    ]
    
    for pattern, display_name, suffix in patterns:
        matching_files = [f for f in estimated_files if pattern in f]
        if matching_files:
            # Try to determine the step name based on common patterns
            if 'prob' in pattern:
                if 'pos' in pattern:
                    step_name = 'positive_probability_model'
                elif 'neg' in pattern:
                    step_name = 'negative_probability_model'
                else:
                    step_name = 'probability_model'
            else:  # level models
                if 'pos' in pattern:
                    step_name = 'positive_level_model'
                elif 'neg' in pattern:
                    step_name = 'negative_level_model'
                else:
                    step_name = 'level_model'
            
            model_types.append((step_name, display_name, suffix))
    
    # Special case for investment models (Tobit)
    tobit_files = [f for f in estimated_files if '_level.json' in f and table_num in [3, 15]]
    if tobit_files and table_num in [3, 15]:
        # Replace level_model with investment_model for these tables
        model_types = [('investment_model', 'Tobit Investment', '_level') if m[0] == 'level_model' else m for m in model_types]
    
    return model_types

def compare_table_coefficients(table_num):
    """Compare all coefficients for a specific table with proper ordering"""
    print(f"{'='*80}")
    print(f"TABLE {table_num} COEFFICIENT COMPARISON")
    print(f"{'='*80}")
    
    # Get table info
    table_info = get_table_info(table_num)
    if not table_info:
        print(f"❌ Table {table_num} not configured")
        return
    
    print(f"📋 {table_info['name']}")
    print(f"🔧 Variables will be displayed in config file order")
    print()
    
    # Import the config
    config_info = get_config_info(table_num)
    if not config_info:
        print(f"❌ Config info not found for table {table_num}")
        return
    
    config_module_name, config_var_name = config_info
    
    try:
        config_module = __import__(config_module_name, fromlist=[config_var_name])
        config_dict = getattr(config_module, config_var_name)
    except (ImportError, AttributeError) as e:
        print(f"❌ Could not import config for table {table_num}: {e}")
        return
    
    # Find estimated files
    estimated_files = find_estimated_files(table_num)
    if not estimated_files:
        print(f"❌ No estimated coefficient files found for table {table_num}")
        return
    
    print(f"📁 Found {len(estimated_files)} estimated coefficient files:")
    for file in estimated_files:
        print(f"   • {file}")
    print()
    
    # Get model types - use configured models or auto-detect
    models = table_info['models']
    if not models:
        print("🔍 Auto-detecting model types from estimated files...")
        models = auto_detect_models(table_num, estimated_files)
        if models:
            print(f"✅ Detected {len(models)} model types: {[m[1] for m in models]}")
        else:
            print("❌ Could not detect any model types")
            return
    
    # Compare each model type with corresponding estimated files
    results_summary = []
    
    for step_name, model_display_name, file_suffix in models:
        print(f"🔍 Analyzing {model_display_name} Model:")
        
        # Find matching estimated file
        matching_files = [f for f in estimated_files if file_suffix in f]
        
        if not matching_files:
            print(f"   ❌ No matching estimated file found for {model_display_name} (looking for: *{file_suffix}*)")
            results_summary.append(f"❌ {model_display_name}: No estimated file")
            continue
        
        for estimated_file in matching_files:
            print(f"   📊 File: {estimated_file}")
            
            comparison = compare_coefficients(
                config_dict=config_dict,
                config_step_name=step_name,
                estimated_json_path=estimated_file,
                model_name=f"Table {table_num} - {model_display_name}"
            )
            
            if comparison is not None:
                display_comparison_table(comparison, f"Table {table_num} - {model_display_name}")
                
                # Add to summary
                total_vars = len(comparison)
                found_vars = (~comparison['is_missing']).sum()
                sign_matches = comparison['same_sign'].sum() if found_vars > 0 else 0
                results_summary.append(f"✅ {model_display_name}: {found_vars}/{total_vars} vars, {sign_matches}/{found_vars} signs match")
            else:
                results_summary.append(f"❌ {model_display_name}: Comparison failed")
            
            print("-" * 60)
    
    # Print table summary
    print(f"📊 TABLE {table_num} SUMMARY:")
    for result in results_summary:
        print(f"   {result}")
    print()

print("✅ Table configuration functions loaded!")
print("📊 Ready for comprehensive analysis of tables 1-24")
print("🎯 All variables will maintain config file ordering")
print("🔧 Auto-detection enabled for tables with missing model configurations")

✅ Table configuration functions loaded!
📊 Ready for comprehensive analysis of tables 1-24
🎯 All variables will maintain config file ordering
🔧 Auto-detection enabled for tables with missing model configurations


## 4. Comprehensive Analysis: Tables 1-24 with Statistical Analysis

Run comprehensive coefficient comparison for all tables. Variables are displayed in the same order as they appear in the config files.

**Analysis includes:**
- All 24 tables (1-24) with their respective model types
- Config file order preservation
- Special handling for Intercept lists
- Auto-detection of model types for tables with incomplete configurations
- Sign agreement and magnitude difference analysis
- **Enhanced Statistical Analysis:**
  - **t-test models**: Std. Error, CI Lower, CI Upper, Config in CI
  - **Chi-square models**: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)
- Summary statistics for each table including statistical data availability

**Coverage:**
- Tables 1-24: Economic depreciation, assets, liabilities, income, expenses, and financial metrics
- Multiple model types: Probability, Level, Tobit, Multinomial models
- Positive/Negative model specifications where applicable
- **Statistical Format Detection**: Automatically detects and displays appropriate statistical measures

**Statistical Features:**
- **Level Models**: Display confidence intervals, standard errors, and coverage indicators for config values
- **Probability Models**: Display Chi-square test statistics, p-values, and significance indicators (p<0.05)
- **Missing Data Handling**: Gracefully handles cases where statistical data is unavailable

In [20]:
# ============================================================================
# RUN COMPREHENSIVE COEFFICIENT COMPARISON FOR ALL TABLES 1-24
# ============================================================================

print("🚀 Starting comprehensive coefficient comparison for Tables 1-24...")
print("🎯 Variables will be displayed in config file order")
print("🔧 Auto-detection enabled for incomplete model configurations")
print("="*80)
print()

# Track overall results
all_results = []
config_errors = []
file_errors = []
success_count = 0

# You can modify this range to test specific tables
# For full analysis: range(1, 25)
# For quick test: range(1, 4) or [1, 5, 10, 15, 20, 24] for specific tables
test_tables = range(1, 25)  # Full analysis - change to test subset if needed

for table_num in test_tables:
    try:
        print(f"Processing Table {table_num}...")
        compare_table_coefficients(table_num)
        all_results.append(f"✅ Table {table_num}: Successfully analyzed")
        success_count += 1
    except ImportError as e:
        error_msg = f"Config import error - {str(e)}"
        print(f"❌ Error processing Table {table_num}: {error_msg}")
        all_results.append(f"❌ Table {table_num}: {error_msg}")
        config_errors.append(table_num)
        print()
    except FileNotFoundError as e:
        error_msg = f"File not found - {str(e)}"
        print(f"❌ Error processing Table {table_num}: {error_msg}")
        all_results.append(f"❌ Table {table_num}: {error_msg}")
        file_errors.append(table_num)
        print()
    except Exception as e:
        error_msg = f"Unexpected error - {str(e)}"
        print(f"❌ Error processing Table {table_num}: {error_msg}")
        all_results.append(f"❌ Table {table_num}: {error_msg}")
        print()

print("="*80)
print("🏁 COMPREHENSIVE ANALYSIS COMPLETE")
print("="*80)
print()
print("📊 OVERALL SUMMARY:")
for result in all_results:
    print(f"   {result}")

print()
print("✅ Analysis completed!")
print("📋 All variables displayed in original config file order")
print("🎯 Ready for detailed coefficient review")

# Detailed statistics summary
total_tables = len(test_tables)
error_count = len(config_errors) + len(file_errors)

print()
print(f"📈 DETAILED STATS:")
print(f"   • Total tables processed: {total_tables}")
print(f"   • Successfully analyzed: {success_count}")
print(f"   • Config import errors: {len(config_errors)} {config_errors if config_errors else ''}")
print(f"   • File not found errors: {len(file_errors)} {file_errors if file_errors else ''}")
print(f"   • Other errors: {error_count - len(config_errors) - len(file_errors)}")
print(f"   • Success rate: {success_count/total_tables*100:.1f}%")

if success_count == total_tables:
    print("🎉 Perfect! All tables analyzed successfully!")
elif success_count >= total_tables * 0.8:
    print("🎯 Great! Most tables analyzed successfully!")
else:
    print("⚠️  Some tables need attention - check error details above")

🚀 Starting comprehensive coefficient comparison for Tables 1-24...
🎯 Variables will be displayed in config file order
🔧 Auto-detection enabled for incomplete model configurations

Processing Table 1...
TABLE 1 COEFFICIENT COMPARISON
📋 EDEPMA (Economic Depreciation MA)
🔧 Variables will be displayed in config file order

📁 Found 2 estimated coefficient files:
   • estimators/coef/t1_edepma_level.json
   • estimators/coef/t1_edepma_prob.json

🔍 Analyzing Probability Model:
   📊 File: estimators/coef/t1_edepma_prob.json
📊 TABLE 1 - PROBABILITY - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)



,Variable,Config Value,Estimated Value,Sign Match,Magnitude Diff,Std. Error,Chi-square,Pr(>ChiSq),Significant
0,Intercept,0.374900,0.363228,✅ Same,-0.0 orders,0.059305,37.512589,9.082e-10,✅ Yes
1,sumcasht_1,-4.800e-11,-4.044e-10,✅ Same,+0.9 orders,1.521e-10,7.063513,0.007867,✅ Yes
2,diffcasht_1,-2.680e-11,-4.493e-11,✅ Same,+0.2 orders,5.730e-11,0.614655,0.433040,❌ No
3,TDEPMAt_1,7.860e-10,2.828e-08,✅ Same,+1.6 orders,8.802e-09,10.326180,0.001312,✅ Yes
4,MAt_1,8.500e-11,1.665e-08,✅ Same,+2.3 orders,1.808e-09,84.759605,0.000e+00,✅ Yes
5,I_MAt_1,1.556e-09,3.924e-08,✅ Same,+1.4 orders,3.524e-09,124.029396,0.000e+00,✅ Yes
6,I_MAt_12,-4.800e-19,-2.641e-15,✅ Same,+3.7 orders,2.324e-16,129.165436,0.000e+00,✅ Yes
7,EDEPBUt_1,3.043e-08,4.368e-08,✅ Same,+0.2 orders,6.060e-08,0.519554,0.471032,❌ No
8,EDEPBUt_12,-1.160e-16,3.392e-14,❌ Different,+2.5 orders,5.666e-14,0.358347,0.549426,❌ No
9,ddmtdmt_1,2.520e-12,3.121e-09,✅ Same,+3.1 orders,8.014e-09,0.151639,0.696974,❌ No


📊 Summary:
   • 22/22 variables found in estimated output
   • 16/22 variables have matching signs
   • 22/22 variables have statistical data (chi-square format)
   • 14/22 variables are significant (p<0.05)

------------------------------------------------------------
🔍 Analyzing Level Model:
   📊 File: estimators/coef/t1_edepma_level.json
📊 TABLE 1 - LEVEL - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including t-test statistics: Std. Error, CI Lower, CI Upper, Config in CI



,Variable,Config Value,Estimated Value,Sign Match,Magnitude Diff,Std. Error,CI Lower,CI Upper,Config in CI
0,Intercept,374143.000000,375641.687500,✅ Same,+0.0 orders,201.855270,375246.062500,376037.312500,❌ No
1,sumcasht_1,2.800e-05,2.813e-05,✅ Same,+0.0 orders,9.317e-08,2.794e-05,2.831e-05,✅ Yes
2,diffcasht_1,-3.000e-05,-3.004e-05,✅ Same,+0.0 orders,3.457e-08,-3.010e-05,-2.997e-05,✅ Yes
3,TDEPMAt_1,0.541900,0.541904,✅ Same,+0.0 orders,4.906e-06,0.541894,0.541913,✅ Yes
4,MAt_1,0.028800,0.028801,✅ Same,+0.0 orders,9.903e-07,0.028799,0.028803,✅ Yes
5,I_MAt_1,0.056300,0.056303,✅ Same,+0.0 orders,1.985e-06,0.056299,0.056307,✅ Yes
6,I_MAt_12,-2.320e-11,-2.331e-11,✅ Same,+0.0 orders,1.311e-13,-2.357e-11,-2.305e-11,✅ Yes
7,EDEPBUt_1,-0.001060,-0.001087,✅ Same,+0.0 orders,3.423e-05,-0.001154,-0.001020,✅ Yes
8,EDEPBUt_12,1.740e-10,1.811e-10,✅ Same,+0.0 orders,3.128e-11,1.198e-10,2.425e-10,✅ Yes
9,ddmtdmt_1,-6.500e-04,-6.423e-04,✅ Same,-0.0 orders,4.676e-06,-6.514e-04,-6.331e-04,✅ Yes


📊 Summary:
   • 22/22 variables found in estimated output
   • 19/22 variables have matching signs
   • 22/22 variables have statistical data (t-test format)
   • 14/22 config values are within confidence intervals

------------------------------------------------------------
📊 TABLE 1 SUMMARY:
   ✅ Probability: 22/22 vars, 16/22 signs match
   ✅ Level: 22/22 vars, 19/22 signs match

Processing Table 2...
TABLE 2 COEFFICIENT COMPARISON
📋 SMA (Selling & Marketing Assets)
🔧 Variables will be displayed in config file order

📁 Found 3 estimated coefficient files:
   • estimators/coef/t2_sma_level_neg.json
   • estimators/coef/t2_sma_level_pos.json
   • estimators/coef/t2_sma_prob.json

🔍 Analyzing Multinomial Probability Model:
   📊 File: estimators/coef/t2_sma_prob.json
📊 TABLE 2 - MULTINOMIAL PROBABILITY - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)



...

📊 Summary:
   • 13/13 variables found in estimated output
   • 9/13 variables have matching signs
   • 13/13 variables have statistical data (t-test format)
   • 0/13 config values are within confidence intervals

------------------------------------------------------------
📊 TABLE 23 SUMMARY:
   ✅ Level: 13/13 vars, 9/13 signs match

Processing Table 24...
TABLE 24 COEFFICIENT COMPARISON
📋 ROT (Return on Total Assets)
🔧 Variables will be displayed in config file order

📁 Found 2 estimated coefficient files:
   • estimators/coef/t24_rot_level.json
   • estimators/coef/t24_rot_prob.json

🔍 Analyzing Probability Model:
   📊 File: estimators/coef/t24_rot_prob.json
📊 TABLE 24 - PROBABILITY - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)



,Variable,Config Value,Estimated Value,Sign Match,Magnitude Diff,Std. Error,Chi-square,Pr(>ChiSq),Significant
0,Intercept,-4.392200,-4.266434,✅ Same,-0.0 orders,0.125914,1148.102905,0.000e+00,✅ Yes
1,sumallozpft,1.847e-08,6.096e-08,✅ Same,+0.5 orders,3.688e-08,2.731351,0.098396,❌ No
2,diffallozpft,-1.300e-08,-6.077e-08,✅ Same,+0.7 orders,4.095e-08,2.202635,0.137775,❌ No
3,TDEPMAt,1.514e-09,-5.561e-08,❌ Different,+1.6 orders,6.237e-08,0.795182,0.372538,❌ No
4,TDEPMAt2,-7.110e-19,2.538e-15,❌ Different,+3.5 orders,1.035e-14,0.060163,0.806238,❌ No
5,OIBDt,1.024e-09,-1.366e-08,❌ Different,+1.1 orders,2.756e-08,0.245729,0.620099,❌ No
6,OIBDt2,-1.580e-19,7.363e-15,❌ Different,+4.6 orders,2.914e-15,6.384336,0.011513,✅ Yes
7,EDEPBUt,-8.130e-10,2.058e-07,❌ Different,+2.4 orders,3.954e-07,0.270958,0.602690,❌ No
8,EDEPBUt2,4.720e-17,-2.870e-13,❌ Different,+3.8 orders,3.413e-13,0.706815,0.400503,❌ No
9,OTAt,-1.180e-10,-1.189e-08,✅ Same,+2.0 orders,1.026e-08,1.342877,0.246527,❌ No


📊 Summary:
   • 24/24 variables found in estimated output
   • 17/24 variables have matching signs
   • 24/24 variables have statistical data (chi-square format)
   • 5/24 variables are significant (p<0.05)

------------------------------------------------------------
🔍 Analyzing Level Model:
   📊 File: estimators/coef/t24_rot_level.json
📊 TABLE 24 - LEVEL - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)



,Variable,Config Value,Estimated Value,Sign Match,Magnitude Diff,Std. Error,Chi-square,Pr(>ChiSq),Significant
0,Intercept,-143891.000000,-3.207e+09,✅ Same,+4.3 orders,47.285625,4.599e+15,0.000e+00,✅ Yes
1,sumallozpft,5.760e-04,18.948881,✅ Same,+4.5 orders,2.898e-05,4.275e+11,0.000e+00,✅ Yes
2,diffallozpft,-0.015200,-339.737854,✅ Same,+4.3 orders,2.648e-05,1.646e+14,0.000e+00,✅ Yes
3,OIBDt,-0.004750,-103.581474,✅ Same,+4.3 orders,6.998e-06,2.191e+14,0.000e+00,✅ Yes
4,EDEPBUt,0.068400,1504.140015,✅ Same,+4.3 orders,4.062e-05,1.371e+15,0.000e+00,✅ Yes
5,TDEPBUt,-0.138200,-3041.121094,✅ Same,+4.3 orders,9.933e-06,9.373e+16,0.000e+00,✅ Yes
6,dourt,0.014100,305.917023,✅ Same,+4.3 orders,1.672e-05,3.349e+14,0.000e+00,✅ Yes
7,TLt,0.031700,693.335999,✅ Same,+4.3 orders,2.885e-05,5.774e+14,0.000e+00,✅ Yes
8,FIt,-4.900e-04,-13.321479,✅ Same,+4.4 orders,3.793e-06,1.233e+13,0.000e+00,✅ Yes
9,FEt,0.061100,1342.416382,✅ Same,+4.3 orders,6.069e-06,4.893e+16,0.000e+00,✅ Yes


📊 Summary:
   • 17/17 variables found in estimated output
   • 16/17 variables have matching signs
   • 17/17 variables have statistical data (chi-square format)
   • 17/17 variables are significant (p<0.05)

------------------------------------------------------------
📊 TABLE 24 SUMMARY:
   ✅ Probability: 24/24 vars, 17/24 signs match
   ✅ Level: 17/17 vars, 16/17 signs match

🏁 COMPREHENSIVE ANALYSIS COMPLETE

📊 OVERALL SUMMARY:
   ✅ Table 1: Successfully analyzed
   ✅ Table 2: Successfully analyzed
   ✅ Table 3: Successfully analyzed
   ✅ Table 4: Successfully analyzed
   ✅ Table 5: Successfully analyzed
   ✅ Table 6: Successfully analyzed
   ✅ Table 7: Successfully analyzed
   ✅ Table 8: Successfully analyzed
   ✅ Table 9: Successfully analyzed
   ✅ Table 10: Successfully analyzed
   ✅ Table 11: Successfully analyzed
   ✅ Table 12: Successfully analyzed
   ✅ Table 13: Successfully analyzed
   ✅ Table 14: Successfully analyzed
   ✅ Table 15: Successfully analyzed
   ✅ Table 16: Succ

## 5. Quick Tests (Optional)

Test individual tables or specific features as needed.

In [21]:
# Test a specific table (change table number as needed)
test_table = 1  # Try Table 1 (EDEPMA) as an example

print(f"🧪 Testing Table {test_table} individually...")
print(f"🎯 Variables will be displayed in config file order")
print()

try:
    compare_table_coefficients(test_table)
    print(f"✅ Table {test_table} test completed successfully!")
except Exception as e:
    print(f"❌ Error testing Table {test_table}: {e}")
    import traceback
    traceback.print_exc()

🧪 Testing Table 1 individually...
🎯 Variables will be displayed in config file order

TABLE 1 COEFFICIENT COMPARISON
📋 EDEPMA (Economic Depreciation MA)
🔧 Variables will be displayed in config file order

📁 Found 2 estimated coefficient files:
   • estimators/coef/t1_edepma_level.json
   • estimators/coef/t1_edepma_prob.json

🔍 Analyzing Probability Model:
   📊 File: estimators/coef/t1_edepma_prob.json
📊 TABLE 1 - PROBABILITY - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)



,Variable,Config Value,Estimated Value,Sign Match,Magnitude Diff,Std. Error,Chi-square,Pr(>ChiSq),Significant
0,Intercept,0.374900,0.363228,✅ Same,-0.0 orders,0.059305,37.512589,9.082e-10,✅ Yes
1,sumcasht_1,-4.800e-11,-4.044e-10,✅ Same,+0.9 orders,1.521e-10,7.063513,0.007867,✅ Yes
2,diffcasht_1,-2.680e-11,-4.493e-11,✅ Same,+0.2 orders,5.730e-11,0.614655,0.433040,❌ No
3,TDEPMAt_1,7.860e-10,2.828e-08,✅ Same,+1.6 orders,8.802e-09,10.326180,0.001312,✅ Yes
4,MAt_1,8.500e-11,1.665e-08,✅ Same,+2.3 orders,1.808e-09,84.759605,0.000e+00,✅ Yes
5,I_MAt_1,1.556e-09,3.924e-08,✅ Same,+1.4 orders,3.524e-09,124.029396,0.000e+00,✅ Yes
6,I_MAt_12,-4.800e-19,-2.641e-15,✅ Same,+3.7 orders,2.324e-16,129.165436,0.000e+00,✅ Yes
7,EDEPBUt_1,3.043e-08,4.368e-08,✅ Same,+0.2 orders,6.060e-08,0.519554,0.471032,❌ No
8,EDEPBUt_12,-1.160e-16,3.392e-14,❌ Different,+2.5 orders,5.666e-14,0.358347,0.549426,❌ No
9,ddmtdmt_1,2.520e-12,3.121e-09,✅ Same,+3.1 orders,8.014e-09,0.151639,0.696974,❌ No


📊 Summary:
   • 22/22 variables found in estimated output
   • 16/22 variables have matching signs
   • 22/22 variables have statistical data (chi-square format)
   • 14/22 variables are significant (p<0.05)

------------------------------------------------------------
🔍 Analyzing Level Model:
   📊 File: estimators/coef/t1_edepma_level.json
📊 TABLE 1 - LEVEL - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including t-test statistics: Std. Error, CI Lower, CI Upper, Config in CI



,Variable,Config Value,Estimated Value,Sign Match,Magnitude Diff,Std. Error,CI Lower,CI Upper,Config in CI
0,Intercept,374143.000000,375641.687500,✅ Same,+0.0 orders,201.855270,375246.062500,376037.312500,❌ No
1,sumcasht_1,2.800e-05,2.813e-05,✅ Same,+0.0 orders,9.317e-08,2.794e-05,2.831e-05,✅ Yes
2,diffcasht_1,-3.000e-05,-3.004e-05,✅ Same,+0.0 orders,3.457e-08,-3.010e-05,-2.997e-05,✅ Yes
3,TDEPMAt_1,0.541900,0.541904,✅ Same,+0.0 orders,4.906e-06,0.541894,0.541913,✅ Yes
4,MAt_1,0.028800,0.028801,✅ Same,+0.0 orders,9.903e-07,0.028799,0.028803,✅ Yes
5,I_MAt_1,0.056300,0.056303,✅ Same,+0.0 orders,1.985e-06,0.056299,0.056307,✅ Yes
6,I_MAt_12,-2.320e-11,-2.331e-11,✅ Same,+0.0 orders,1.311e-13,-2.357e-11,-2.305e-11,✅ Yes
7,EDEPBUt_1,-0.001060,-0.001087,✅ Same,+0.0 orders,3.423e-05,-0.001154,-0.001020,✅ Yes
8,EDEPBUt_12,1.740e-10,1.811e-10,✅ Same,+0.0 orders,3.128e-11,1.198e-10,2.425e-10,✅ Yes
9,ddmtdmt_1,-6.500e-04,-6.423e-04,✅ Same,-0.0 orders,4.676e-06,-6.514e-04,-6.331e-04,✅ Yes


📊 Summary:
   • 22/22 variables found in estimated output
   • 19/22 variables have matching signs
   • 22/22 variables have statistical data (t-test format)
   • 14/22 config values are within confidence intervals

------------------------------------------------------------
📊 TABLE 1 SUMMARY:
   ✅ Probability: 22/22 vars, 16/22 signs match
   ✅ Level: 22/22 vars, 19/22 signs match

✅ Table 1 test completed successfully!


In [22]:
# Test configuration accessibility for all tables 1-24
print("🔧 Testing config accessibility for all tables...")

available_configs = []
missing_configs = []

for table_num in range(1, 25):
    config_info = get_config_info(table_num)
    if config_info:
        config_module_name, config_var_name = config_info
        try:
            config_module = __import__(config_module_name, fromlist=[config_var_name])
            config_dict = getattr(config_module, config_var_name)
            available_configs.append(table_num)
            
            # Quick check for number of steps
            steps = len(config_dict.get('steps', []))
            if table_num <= 5:  # Show details for first few tables
                table_info = get_table_info(table_num)
                print(f"✅ Table {table_num} ({table_info['name']}): {steps} steps")
        except (ImportError, AttributeError) as e:
            missing_configs.append(table_num)
            print(f"❌ Table {table_num}: Config import failed - {e}")
    else:
        missing_configs.append(table_num)
        print(f"❌ Table {table_num}: No config info defined")

print()
print(f"📊 Configuration Summary:")
print(f"   • Available configs: {len(available_configs)}/24 tables")
print(f"   • Missing configs: {missing_configs if missing_configs else 'None'}")

# Quick estimated files check
print()
print("📁 Checking estimated coefficient files...")
total_files = 0
for table_num in range(1, 25):
    files = find_estimated_files(table_num)
    total_files += len(files)
    if table_num <= 5:  # Show details for first few tables
        print(f"   Table {table_num}: {len(files)} files")

print(f"   • Total estimated files found: {total_files}")

print()
print("✅ Configuration test completed!")
print(f"🎯 Ready to analyze {len(available_configs)} tables with full configs")

🔧 Testing config accessibility for all tables...
✅ Table 1 (EDEPMA (Economic Depreciation MA)): 2 steps
✅ Table 2 (SMA (Selling & Marketing Assets)): 3 steps
✅ Table 3 (IMA (Investment Model Assets)): 1 steps
✅ Table 4 (EDEPBU (Economic Depreciation BU)): 2 steps
✅ Table 5 (IBU (Investment in Business Use)): 2 steps

📊 Configuration Summary:
   • Available configs: 24/24 tables
   • Missing configs: None

📁 Checking estimated coefficient files...
   Table 1: 2 files
   Table 2: 3 files
   Table 3: 1 files
   Table 4: 2 files
   Table 5: 2 files
   • Total estimated files found: 49

✅ Configuration test completed!
🎯 Ready to analyze 24 tables with full configs
   Table 2: 3 files
   Table 3: 1 files
   Table 4: 2 files
   Table 5: 2 files
   • Total estimated files found: 49

✅ Configuration test completed!
🎯 Ready to analyze 24 tables with full configs


In [23]:
# Enhanced sample analysis with statistical features demonstration
print("🎯 Running enhanced sample analysis with statistical features...")
print("📊 Demonstrating the new statistical analysis capabilities")
print("📈 Shows both t-test and Chi-square statistics formats")
print()

# Sample tables that showcase different statistical formats
sample_tables = [1, 10]  # Table 1 for both formats, Table 10 for complex models

for table_num in sample_tables:
    try:
        print(f"📋 Enhanced analysis of Table {table_num}...")
        
        # Get basic info
        table_info = get_table_info(table_num)
        estimated_files = find_estimated_files(table_num)
        
        print(f"   • Name: {table_info['name']}")
        print(f"   • Models: {len(table_info['models'])} configured")
        print(f"   • Estimated files: {len(estimated_files)} found")
        
        # Check for statistical data in files
        stats_summary = []
        for file in estimated_files[:2]:  # Check first 2 files
            try:
                with open(file, 'r') as f:
                    data = json.load(f)
                
                stats_data = data.get('statistics', {})
                if 'coefficient_stats' in stats_data:
                    stats_summary.append(f"Chi-square stats in {file.split('/')[-1]}")
                elif isinstance(stats_data, list) and stats_data:
                    stats_summary.append(f"t-test stats in {file.split('/')[-1]}")
                else:
                    stats_summary.append(f"No stats in {file.split('/')[-1]}")
            except:
                stats_summary.append(f"Error reading {file.split('/')[-1]}")
        
        for summary in stats_summary:
            print(f"   📈 {summary}")
        
        print()
        
    except Exception as e:
        print(f"   ❌ Error with Table {table_num}: {e}")
        print()

print("✅ Enhanced sample analysis completed!")
print("🚀 All tables 1-24 are ready for comprehensive analysis with statistical measures")
print("📈 Statistical features:")
print("   • t-test: Std. Error, CI Lower, CI Upper")
print("   • Chi-square: Std. Error, Chi-square, Pr(>ChiSq)")
print("   • Automatic format detection and display")

🎯 Running enhanced sample analysis with statistical features...
📊 Demonstrating the new statistical analysis capabilities
📈 Shows both t-test and Chi-square statistics formats

📋 Enhanced analysis of Table 1...
   • Name: EDEPMA (Economic Depreciation MA)
   • Models: 2 configured
   • Estimated files: 2 found
   📈 t-test stats in t1_edepma_level.json
   📈 Chi-square stats in t1_edepma_prob.json

📋 Enhanced analysis of Table 10...
   • Name: DSC (Debt Service Charges)
   • Models: 4 configured
   • Estimated files: 4 found
   📈 t-test stats in t10_dsc_level_neg.json
   📈 t-test stats in t10_dsc_level_pos.json

✅ Enhanced sample analysis completed!
🚀 All tables 1-24 are ready for comprehensive analysis with statistical measures
📈 Statistical features:
   • t-test: Std. Error, CI Lower, CI Upper
   • Chi-square: Std. Error, Chi-square, Pr(>ChiSq)
   • Automatic format detection and display


In [24]:
# ============================================================================
# DEMONSTRATION: Enhanced Statistical Features
# ============================================================================

print("🧪 DEMONSTRATION: Statistical Features")
print("="*60)
print("📈 Testing enhanced visualization with statistical analysis")
print("🎯 Showing both t-test and Chi-square formats")
print()

# Test Table 1 which has both probability (Chi-square) and level (t-test) models
test_table = 1

print(f"🔍 Testing Table {test_table} (EDEPMA) - Shows both statistical formats:")
print()

try:
    # This will demonstrate both Chi-square (probability) and t-test (level) statistics
    compare_table_coefficients(test_table)
    
    print("✅ Statistical feature demonstration completed!")
    print()
    print("📊 What you should see above:")
    print("   • Probability model with Chi-square statistics (Std. Error, Chi-square, Pr(>ChiSq), Significant)")
    print("   • Level model with t-test statistics (Std. Error, CI Lower, CI Upper)")
    print("   • Variables displayed in config file order")
    print("   • Summary includes statistical data availability")
    
except Exception as e:
    print(f"❌ Error in demonstration: {e}")
    import traceback
    traceback.print_exc()

print()
print("📈 Statistical Integration Features:")
print("   ✅ Chi-square format auto-detection")
print("   ✅ t-test format auto-detection") 
print("   ✅ Appropriate statistical measures displayed")
print("   ✅ Significance indicators for Chi-square tests (p<0.05)")
print("   ✅ Missing statistical data handled gracefully")
print("   ✅ Enhanced summary with statistical coverage")

🧪 DEMONSTRATION: Statistical Features
📈 Testing enhanced visualization with statistical analysis
🎯 Showing both t-test and Chi-square formats

🔍 Testing Table 1 (EDEPMA) - Shows both statistical formats:

TABLE 1 COEFFICIENT COMPARISON
📋 EDEPMA (Economic Depreciation MA)
🔧 Variables will be displayed in config file order

📁 Found 2 estimated coefficient files:
   • estimators/coef/t1_edepma_level.json
   • estimators/coef/t1_edepma_prob.json

🔍 Analyzing Probability Model:
   📊 File: estimators/coef/t1_edepma_prob.json
📊 TABLE 1 - PROBABILITY - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including Chi-square statistics: Std. Error, Chi-square, Pr(>ChiSq), Significant (p<0.05)



,Variable,Config Value,Estimated Value,Sign Match,Magnitude Diff,Std. Error,Chi-square,Pr(>ChiSq),Significant
0,Intercept,0.374900,0.363228,✅ Same,-0.0 orders,0.059305,37.512589,9.082e-10,✅ Yes
1,sumcasht_1,-4.800e-11,-4.044e-10,✅ Same,+0.9 orders,1.521e-10,7.063513,0.007867,✅ Yes
2,diffcasht_1,-2.680e-11,-4.493e-11,✅ Same,+0.2 orders,5.730e-11,0.614655,0.433040,❌ No
3,TDEPMAt_1,7.860e-10,2.828e-08,✅ Same,+1.6 orders,8.802e-09,10.326180,0.001312,✅ Yes
4,MAt_1,8.500e-11,1.665e-08,✅ Same,+2.3 orders,1.808e-09,84.759605,0.000e+00,✅ Yes
5,I_MAt_1,1.556e-09,3.924e-08,✅ Same,+1.4 orders,3.524e-09,124.029396,0.000e+00,✅ Yes
6,I_MAt_12,-4.800e-19,-2.641e-15,✅ Same,+3.7 orders,2.324e-16,129.165436,0.000e+00,✅ Yes
7,EDEPBUt_1,3.043e-08,4.368e-08,✅ Same,+0.2 orders,6.060e-08,0.519554,0.471032,❌ No
8,EDEPBUt_12,-1.160e-16,3.392e-14,❌ Different,+2.5 orders,5.666e-14,0.358347,0.549426,❌ No
9,ddmtdmt_1,2.520e-12,3.121e-09,✅ Same,+3.1 orders,8.014e-09,0.151639,0.696974,❌ No


📊 Summary:
   • 22/22 variables found in estimated output
   • 16/22 variables have matching signs
   • 22/22 variables have statistical data (chi-square format)
   • 14/22 variables are significant (p<0.05)

------------------------------------------------------------
🔍 Analyzing Level Model:
   📊 File: estimators/coef/t1_edepma_level.json
📊 TABLE 1 - LEVEL - Coefficient Comparison:
   Variables ordered as they appear in config file
   📈 Including t-test statistics: Std. Error, CI Lower, CI Upper, Config in CI



,Variable,Config Value,Estimated Value,Sign Match,Magnitude Diff,Std. Error,CI Lower,CI Upper,Config in CI
0,Intercept,374143.000000,375641.687500,✅ Same,+0.0 orders,201.855270,375246.062500,376037.312500,❌ No
1,sumcasht_1,2.800e-05,2.813e-05,✅ Same,+0.0 orders,9.317e-08,2.794e-05,2.831e-05,✅ Yes
2,diffcasht_1,-3.000e-05,-3.004e-05,✅ Same,+0.0 orders,3.457e-08,-3.010e-05,-2.997e-05,✅ Yes
3,TDEPMAt_1,0.541900,0.541904,✅ Same,+0.0 orders,4.906e-06,0.541894,0.541913,✅ Yes
4,MAt_1,0.028800,0.028801,✅ Same,+0.0 orders,9.903e-07,0.028799,0.028803,✅ Yes
5,I_MAt_1,0.056300,0.056303,✅ Same,+0.0 orders,1.985e-06,0.056299,0.056307,✅ Yes
6,I_MAt_12,-2.320e-11,-2.331e-11,✅ Same,+0.0 orders,1.311e-13,-2.357e-11,-2.305e-11,✅ Yes
7,EDEPBUt_1,-0.001060,-0.001087,✅ Same,+0.0 orders,3.423e-05,-0.001154,-0.001020,✅ Yes
8,EDEPBUt_12,1.740e-10,1.811e-10,✅ Same,+0.0 orders,3.128e-11,1.198e-10,2.425e-10,✅ Yes
9,ddmtdmt_1,-6.500e-04,-6.423e-04,✅ Same,-0.0 orders,4.676e-06,-6.514e-04,-6.331e-04,✅ Yes


📊 Summary:
   • 22/22 variables found in estimated output
   • 19/22 variables have matching signs
   • 22/22 variables have statistical data (t-test format)
   • 14/22 config values are within confidence intervals

------------------------------------------------------------
📊 TABLE 1 SUMMARY:
   ✅ Probability: 22/22 vars, 16/22 signs match
   ✅ Level: 22/22 vars, 19/22 signs match

✅ Statistical feature demonstration completed!

📊 What you should see above:
   • Probability model with Chi-square statistics (Std. Error, Chi-square, Pr(>ChiSq), Significant)
   • Level model with t-test statistics (Std. Error, CI Lower, CI Upper)
   • Variables displayed in config file order
   • Summary includes statistical data availability

📈 Statistical Integration Features:
   ✅ Chi-square format auto-detection
   ✅ t-test format auto-detection
   ✅ Appropriate statistical measures displayed
   ✅ Significance indicators for Chi-square tests (p<0.05)
   ✅ Missing statistical data handled gracefully
